In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install PyMuPDF
!pip install opencv-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 42 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 0s (12.5 MB/s)
debconf: unable to initia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [63]:
from PIL import Image
import fitz
import re
import pandas as pd
import cv2
import pytesseract
import numpy as np
name = "AK_T2"
pdf = fitz.open(f"{name}.pdf")
datadict = {}

In [64]:
def get_data_from_annots(pdf, regex):
  annot_list = []
  annots = pdf[0].annots(types=fitz.PDF_ANNOT_SQUARE)
  for annot in annots:
    annot_content = annot.get_text(option="text")
    annot_list.append(annot_content) if len(annot_content)>0 else 0

  annot_list = [element.split("\n") for element in annot_list]
  annot_list = [subitem for item in annot_list for subitem in item if len(subitem)>0]

  #2 - Controleer of annotaties de aktereferentiecode bevatten
  reference_code = [element for element in annot_list if re.search(regex,element)] if regex is not None else None
  return reference_code, annot_list

def get_reference_code_from_selectable_text(pdf, regex):
  text_page_1 = pdf[0].get_text()
  reference_code = re.search(regex,text_page_1)[0] if re.search(regex,text_page_1) is not None else None
  if reference_code is None or len(reference_code)<1:
    return None
  return reference_code

def get_string_from_scanned_document(pdf):
  textstring = ""
  max_range = pdf.page_count if pdf.page_count<3 else 3
  for i in range(max_range):
    img_first_page = fitz.Pixmap(pdf, pdf.get_page_images(i)[0][0])
    img_first_page.save("temp_img_text.jpg")
    del img_first_page
    img = cv2.imread("temp_img_text.jpg")
    textstring += pytesseract.image_to_string(img) 
  return textstring

def search_string_for_reference_code(textstring, regex):
  ref_code = re.search(regex, textstring)
  return ref_code

def get_reference_code(pdf):
  reference_code_regex = re.compile("[0-9].-[A-Z]-([0-9].\/)*[0-9]{4}-[0-9]{5}")
  reference_code, _ = get_data_from_annots(pdf, reference_code_regex)
  textstring_pdf = None
  if len(reference_code)==0:
    reference_code = get_reference_code_from_selectable_text(pdf, reference_code_regex)
  if reference_code is None:
    textstring_pdf = get_string_from_scanned_document(pdf)
    reference_code = search_string_for_reference_code(textstring_pdf, reference_code_regex)
  return reference_code, textstring_pdf


################################################################################

def get_image_list(pdf):
  for page_index in range(len(pdf)):
    image_list = pdf[page_index].get_images()
  return len(image_list)

def process_annots(pdf, annots):
  image_count = get_image_list(pdf)
  text = pdf[0].get_text() #Selecteer eerste blz om te doorzoeken op niet-gevonden annotaties.
  if image_count>0:
    return text
  else:
    return [re.search(re.compile("[0-9]{1,}"+annot),text)[0] for annot in annots if re.search(re.compile("[0-9]{1,} "+annot),text) is not None]
    

def main(pdf):
  reference_code, textstring_pdf = get_reference_code(pdf)
  _,annots = get_data_from_annots(pdf,regex=None)

  annots_temp = process_annots(pdf, annots)
  if len(annots_temp)>0:
    annots = annots_temp
  print(reference_code, annots)
  
main(pdf)



63-T-18/05/2020-02213 ['ERKOOP', ' PART ', '1 BLZ ', 'NTSLAG']


In [3]:
#1 - Bepalen of annotaties voldoende zijn
def get_data_from_annots(pdf):
  annot_list = []
  annots = pdf[0].annots(types=fitz.PDF_ANNOT_SQUARE)
  for annot in annots:
    annot_content = annot.get_text(option="text")
    annot_list.append(annot_content) if len(annot_content)>0 else 0

  annot_list = [element.split("\n") for element in annot_list]
  annot_list = [subitem for item in annot_list for subitem in item if len(subitem)>0]

  #2 - Controleer of annotaties de aktereferentiecode bevatten
  ref_regex = re.compile("[0-9].-[A-Z]-([0-9].\/)*[0-9]{4}-[0-9]{5}")
  reference_code = [element for element in annot_list if re.search(ref_regex,element)]

  if len(reference_code)<1:
    #Geen referentiecode gedetecteerd in annots
    text_page_1 = pdf[0].get_text()
    reference_code = re.search(ref_regex,text_page_1)[0]
    if len(reference_code)<1:
      return None
  return reference_code, annot_list

#3 - Extraheren uit selecteerbare tekst
def get_data_from_selectable_text(pdf, get_ref_code=False):
  if isinstance(pdf,str) == False:
    textstring = pdf[0].get_text()
  else:
    textstring = pdf
  try:
    textstring = textstring.replace(","," ").replace(".","\next").replace("?","\next").replace("\n","\next").replace('"',' ').replace("!","\next").split("\next")
  except:
    return None
  worddict={}
  if get_ref_code:
    ref_regex = re.compile("[0-9].-[A-Z]-([0-9].\/)*[0-9]{4}-[0-9]{5}")
    return re.search(ref_regex, textstring)
  for word in textstring:
    if word in worddict.keys():
      worddict[word] += 1
    else:
      worddict[word] = 1

  reg_col = re.compile("([0-9]{1,}){0,}.(?:[A-Z]{3,})")
  df = pd.DataFrame.from_dict(worddict, orient="index").sort_values(by=0).T
  necessary_cols = [col for col in df.columns if re.search(reg_col,col)]
  df = df.loc[:,necessary_cols]
  df.columns = [re.search(reg_col,col)[0].strip() for col in df.columns]
  df = df.loc[:,~df.columns.duplicated()]
  return df

#4 - Via OCR-extraheren
def get_data_from_ocr(pdf, get_ref_code=False):
  textstring = ""
  max_range = pdf.page_count if get_ref_code else pdf.page_count if pdf.page_count<3 else 3
  for i in range(max_range):
    img_first_page = fitz.Pixmap(pdf, pdf.get_page_images(i)[0][0])
    img_first_page.save("temp_img_text.jpg")
    del img_first_page
    img = cv2.imread("temp_img_text.jpg")
    textstring += pytesseract.image_to_string(img) 
  if get_ref_code:
    ref_regex = re.compile("[0-9].-[A-Z]-([0-9].\/)*[0-9]{4}-[0-9]{5}")
    return re.search(ref_regex, textstring)
  text1_worddf = get_data_from_selectable_text(textstring)
  return list(text1_worddf.columns)

def get_reference_code_from_text(pdf):
  ref = get_data_from_selectable_text(pdf, True)
  if ref is None:
    None
  else:
    return ref[0]

reference_code, annot_list = get_data_from_annots(pdf)
if reference_code is None:
  reference_code = get_reference_code_from_text(pdf)
  if reference_code is None:
    datadict["Reference Code"] = "Error - not detected within document."
  else:
    datadict["Reference Code"] = reference_code
else:
  if isinstance(reference_code,str)==False:
    reference_code = reference_code[0]
  datadict["Reference Code"] = reference_code
  datadict["Annotations"] = annot_list
  text_data = get_data_from_selectable_text(pdf)
  if text_data is None or len(text_data.columns)<3:
    datadict["Extra data from text"] = get_data_from_ocr(pdf)
  else:
    datadict["Extra data from text"] = list(text_data.columns)

datadict



{'Annotations': ['RECHT VAN OPSTAL', ' BLZ  ', ' PART ', 'BEDRAG'],
 'Extra data from text': ['RECHT', '8 BLZ', '2 PART', 'BEDRAG'],
 'Reference Code': '63-T-30/11/2016-07311'}